# Проект Ford vs. Ferrari определение марки авто по изображению



### Цель: используя предобученные сети и различные техники обучения, построить модель, классифицирующую автомобиль по его изображению

Для достижения поставленной цели были произведены эксперименты, которые включали в себя вариации следующих параметров:

* learning rate = {1e-2, 1e-3, 1e-4}
* image size = {180, 224, 280}
* augmentation = {base, advanced}
* net = {Xception, EfficientNetB7}
* head = {base simple, base, base+batch}
* finetuning = {yes, no}

Ниже преведен код, который показал наилучший результат в соревновании Kaggle. Он содержит описание всех действий, а также некоторые выводы, которые удалось сделать в процессе экспериментов. 

Загрузим efficientnet и библиотеку для продвинутой аугментации изображений:

In [ ]:
!pip install - q efficientnet
!pip install git+https: // github.com/mjkvaak/ImageDataAugmentor

Импортируем необходимые библиотеки:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
import efficientnet.tfkeras as efn
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split

import tensorflow.keras as keras
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
import tensorflow.keras.callbacks as C
from tensorflow.keras.preprocessing import image

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from ImageDataAugmentor.image_data_augmentor import *
import albumentations

import PIL
from PIL import ImageOps, ImageFilter
# увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

Проверим наличие доступа к видеокарточкам и установим необходимые программы из requierements

In [ ]:
!nvidia-smi
!pip freeze > requirements.txt

# Основные настройки

In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS = 10  # эпох на обучение
BATCH_SIZE = 32  # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR = 1e-3
VAL_SPLIT = 0.2  # сколько данных выделяем на тест = 20%

CLASS_NUM = 10  # количество классов в нашей задаче
IMG_SIZE = 280  # какого размера подаем изображения в сеть (180, 224)
IMG_CHANNELS = 3   # у RGB 3 канала
input_shape = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/'
PATH = "../working/car/"  # рабочая директория

# Устанавливаем random seed для воспроизводимости

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
# посмотрим на распределение классов - оно достаточно равномерное
train_df.Category.value_counts()

In [ ]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/"+data_zip, "r") as z:
        z.extractall(PATH)
print(os.listdir(PATH))

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12, 8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3, 3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Как можно видеть - классы это модели автомобилей

Посмотрим на примеры картинок и их размеры, чтобы понимать, как их лучше обработать и сжимать.

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# Подготовка данных

Аугментация данных необходима для того, чтобы иметь возможность качественно обучить сеть при наличии небольшого набора обучающих данных. Аугментация - это процесс применения различных преобразований (сжатие, поворот, отражение и др.) к изображениям. В данной работе применялись два типа аугментаций: base (из бейзлайна) и advanced (библиотека albumentations). Наилучший результат был получен с использованием продвинутой техники аугментации. Были использованы преобразования:
* горизонтальное отражение
* поворот
* изменение яркости и контраста
* обрезка
* наложение Гауссовского фильтра
* изменение оттенка и насыщенности
* сдвиг значения цветов пикселей
* применение метода главных компонент
* изменение размера изображения

Эти преобразования относятся лишь к тренировочным данным, так как тестирование проводится на изображениях, максимально похожих на входные данные, которые встретятся на практике (без аугментации). Кроме этого, значения яркости пикселей всех имеющихся изображений нормируется. 

In [ ]:
# base
# train_datagen = ImageDataGenerator(rescale=1./ 255,
#                                    rotation_range = 15,
#                                    shear_range=0.2,
#                                    zoom_range=[0.75,1.25],
#                                    brightness_range=[0.5, 1.5],
#                                    width_shift_range=0.1,
#                                    height_shift_range=0.1,
#                                    validation_split=VAL_SPLIT, # set validation split
#                                    horizontal_flip=True)

In [ ]:
# advanced
augmentation = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=20, interpolation=1, border_mode=4,
                          value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=224, width=200),
        albumentations.CenterCrop(height=200, width=224),
    ], p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(
            brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(
            brightness_limit=0.1, contrast_limit=0.1)
    ], p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])

train_datagen = ImageDataAugmentor(
    rescale=1./255,
    augment=augmentation,
    validation_split=VAL_SPLIT,
)

test_datagen = ImageDataAugmentor(rescale=1./255)

### Генерация данных

In [ ]:
# Поместим наши данные в генератор:

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training')  # пометим как тренировочные

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation')  # пометим как тестовые

# Построение модели

В качестве базовой сети были рассмотрены два варианта: Xception и EfficientNetB7. Проведенные эксперименты показали, что лучшее качество достигается при использовании EfficientNetB7. Поэтому загрузим её:

In [ ]:
# base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)

In [ ]:
base_model = efn.EfficientNetB7(
    weights='imagenet', include_top=False, input_shape=input_shape)

Посмотрим на устройство базовой модели:

In [ ]:
base_model.summary()

Как показали эксперименты, поэтапное обучение модели, использующее "заморозку" слоев, дает лучшее качество. Это связано с тем, что базовая модель, обученная на imagenet, подстраивается под наши данные постепенно. То есть сначала мы обучаем лишь "голову", оставляя веса базовой модели неизменными, а затем постепенно включаем в обучение слои базовой модели. 

In [ ]:
# зафиксируем базовую модель
base_model.trainable = False

В процессе построения модели были использованы три модификации "головы": base simple, base, base+batch

* base simple = (GlobalAveragePooling2D + Dense)
* base = (GlobalAveragePooling2D + Dense + Dropout + Dense)
* base+batch = (GlobalAveragePooling2D + Dense + BatchNormalization + Dropout + Dense)

Эксперименты показали, что base simple приводит к скорому переобучению модели, хотя и показывает лучший результат на тесте. base и base+batch содержат инструменты для предотвращения переобучения: dropout и batch normalization слои. Лучший результат был получен с использованием батч-нормализации

In [ ]:
# установка "головы"
model = M.Sequential()
model.add(base_model)
model.add(L.GlobalAveragePooling2D())
model.add(L.Dense(256, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.25))
model.add(L.Dense(CLASS_NUM, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    print(layer, layer.trainable)

Можно видеть, что базовая модель "заморожена", а обучаются только слои "головы".

# Обучение модели

### Обучение "головы"

Добавим ModelCheckpoint и EarlyStopping, для того, чтобы иметь возможность сохранять прогресс обучения модели и не совершать лишних вычислений в случае, когда улучшение метрики не происходит

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
checkpoint = ModelCheckpoint('best_model.hdf5', monitor=[
                             'val_accuracy'], verbose=1, mode='max')
earlystop = EarlyStopping(
    monitor='accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

Запустим обучение:

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples//test_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

Посмотрим на значение метрики для тестовых данных:

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Только при обучении "головы" удалось добиться 75% точности

In [ ]:
def draw_curves():
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

In [ ]:
draw_curves()

In [ ]:
# сохраним сеть и загрузим веса лучшего варианта
model.save('../working/step1.hdf5')
model.load_weights('best_model.hdf5')

### Разморозка (первый шаг)

разморозим половину слоев базовой модели и продолжим обучение 

In [ ]:
base_model.trainable = True
fine_tune = len(base_model.layers)//2
for layer in base_model.layers[:fine_tune]:
    layer.trainable = False

In [ ]:
len(base_model.trainable_variables)

In [ ]:
# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

Можно видеть, что базовая модель тоже обучается. Продолжим обучение, при этом уменьшим learning rate до 1e-4

In [ ]:
LR = 1e-4

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples//test_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

Посмотрим на значение метрики на тестовых данных:

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Уже лучше - 95%

In [ ]:
draw_curves()

In [ ]:
# сохраним сеть и загрузим веса лучшего варианта
model.save('../working/step2.hdf5')
model.load_weights('best_model.hdf5')

### Разморозка (второй шаг)

Разморозим всю сеть, предоставив ей возможность обучаться полностью, при этом снова уменьшим learning rate до 1e-5 и batch_size, чтобы сеть поместилась на карту

In [ ]:
base_model.trainable = True
LR = 1e-5
BATCH_SIZE = 10

In [ ]:
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training')  # пометим как тренировочные

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation')  # пометим как тестовые

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
checkpoint = ModelCheckpoint('best_model.hdf5', monitor=[
                             'accuracy'], verbose=1, mode='max')
earlystop = EarlyStopping(
    monitor='accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples//test_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

Посмотрим на значение метрики на тестовых данных:

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Результат при разморозке всей модели - 97.3%

In [ ]:
draw_curves()

In [ ]:
# сохраним сеть и загрузим веса лучшего варианта
model.save('../working/step3.hdf5')
model.load_weights('best_model.hdf5')

### Дополнительно

Одним из приемов улучшения качества модели является увеличение размера изображения совместно с понижением количества преобразований при аугментации. Воспользуемся этим приемом:

In [ ]:
IMG_SIZE = 512
BATCH_SIZE = 2
LR = 1e-5
EPOCHS = 2

In [ ]:
input_shape = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

In [ ]:
augmentation = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=20, interpolation=1, border_mode=4,
                          value=None, mask_value=None, always_apply=False, p=0.5)
])

train_datagen = ImageDataAugmentor(
    rescale=1./255,
    augment=augmentation,
    validation_split=VAL_SPLIT,
)

test_datagen = ImageDataAugmentor(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training')

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation')

Так как размер входных данных изменился, то создадим сеть заново, при этом подгрузим веса, полученные ранее:

In [ ]:
base_model = efn.EfficientNetB7(
    weights='imagenet', include_top=False, input_shape=input_shape)
# установка "головы"
model = M.Sequential()
model.add(base_model)
model.add(L.GlobalAveragePooling2D())
model.add(L.Dense(256, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.25))
model.add(L.Dense(CLASS_NUM, activation='softmax'))

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])
model.load_weights('best_model.hdf5')
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples//test_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

In [ ]:
model.save('../working/step4.hdf5')
model.load_weights('best_model.hdf5')

# Сабмит на Kaggle

Воспользуемся итоговой моделью для предсказания класса автомобиля и сделаем сабмит на Kaggle

In [ ]:
test_sub_generator = test_datagen.flow_from_dataframe(
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

In [ ]:
test_sub_generator.samples

In [ ]:
test_sub_generator.reset()
predictions = model.predict_generator(
    test_sub_generator, steps=len(test_sub_generator), verbose=1)
predictions = np.argmax(predictions, axis=-1)  # multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v, k) for k, v in label_map.items())  # flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir = test_sub_generator.filenames
submission = pd.DataFrame(
    {'Id': filenames_with_dir, 'Category': predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/', '')
submission.to_csv('submission.csv', index=False)
print('Save submit')

In [ ]:
submission.head()

Полученный результат на Kaggle - **0.96374**

# Test Time Augmentation

Еще одной техникой улучшения модели является Test Time Augmentation. Для этого мы аугментируем тестовые изображения и делаем несколько предсказаний. Затем берем среднее от полученных предсказаний, что и будет финальным ответом. 

In [ ]:
model.load_weights('best_model.hdf5')

In [ ]:
augmentation = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=20, interpolation=1, border_mode=4,
                          value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=220, width=200),
        albumentations.CenterCrop(height=200, width=220),
    ], p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(
            brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(
            brightness_limit=0.1, contrast_limit=0.1)
    ], p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])

test_datagen = ImageDataAugmentor(
    rescale=1./255,
    augment=augmentation,
    validation_split=VAL_SPLIT,
)

In [ ]:
test_sub_generator = test_datagen.flow_from_dataframe(
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

In [ ]:
tta_steps = 10
predictions = []

for i in range(tta_steps):
    preds = model.predict_generator(
        test_sub_generator, steps=len(test_sub_generator), verbose=1)
    predictions.append(preds)

pred = np.mean(predictions, axis=0)

In [ ]:
predictions = np.argmax(pred, axis=-1)  # multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v, k) for k, v in label_map.items())  # flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir = test_sub_generator.filenames
submission = pd.DataFrame(
    {'Id': filenames_with_dir, 'Category': predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/', '')
submission.to_csv('submission_TTA.csv', index=False)
print('Save submit')

In [ ]:
# Clean PATH
import shutil
shutil.rmtree(PATH)

TTA дал незначительный прирост качества модели на Kaggle - **0.96958**

# Выводы

В ходе выполнения проекта удалось достичь значение метрики **0.96958**, что является несомненным улучшением baseline.

Применены следующие техники:

* transfer learning
* fine tuning
* TTA

Опробованы сети для классификации изображений:

* Exception 
* EfficientNetB7

Подобраны параметры:

* learning rate
* image size

Подобрана архитектура "головы" и использованы дополнительные callback функции в Keras